## Declare Global Variables

Define batch size, epochs, input shape, and value of k (for k-fold cross validation) here.

In [ ]:
# Define the number of classes
classes = ['ChairPose', 'ChestBump', 'ChildPose', 'Dabbing', 'EaglePose', 'HandGun', 'HandShake', 'HighKneel', 
           'HulkSmash', 'KoreanHeart', 'KungfuCrane', 'KungfuSalute', 'Salute', 'Spiderman', 'WarriorPose']
num_classes = len(classes)

# Batch size
bs = 16

# Number of epochs
epochs = 100

# Image dimensions
img_width = 299
img_height = 299

# All images will be resized to this value
img_size = (img_width, img_height)

# Here we specify the input shape of our data 
# This should match the size of images ('img_size') along with the number of channels (3 -> RGB)
input_shape = (img_width, img_height, 3)

# k-fold cross validation
# Set k=1 for normal train-val split
k = 1

## Retrieve external libraries

2 external libraries are used.
<br>
1) AdaBound for Keras (https://arxiv.org/abs/1902.09843)
<br>
2) Augmentor (http://augmentor.readthedocs.io)

In [ ]:
# AdaBound (optimiser)
!git clone https://github.com/titu1994/keras-adabound.git
!cp keras-adabound/adabound.py .

# Augmentor (data augmentation)
!pip install Augmentor

## Download and Explore the Dataset

The contents of the `.zip` (no longer accessible) are extracted to the base directory, which contains `train` and `val` subdirectories. The folders have the following structure:

```
---------------
train
|- ChairPose
|- ChestBump
|- ChildPose
|- Dabbing
|- EaglePose
|- HandGun
|- HandShake
|- HighKneel
|- HulkSmash
|- KoreanHeart
|- KungfuCrane
|- KungfuSalute
|- Salute
|- Spiderman
|- WarriorPose

val
|- ChairPose
|- ChestBump
|- ChildPose
|- Dabbing
|- EaglePose
|- HandGun
|- HandShake
|- HighKneel
|- HulkSmash
|- KoreanHeart
|- KungfuCrane
|- KungfuSalute
|- Salute
|- Spiderman
|- WarriorPose
---------------
```

In [ ]:
!rm -rf data && rm -rf model && rm -rf graph
# Creating two directories - "data" and "data/trainset_11classes_0_00000"
!mkdir data && mkdir data/trainset_11classes_0_00000 && mkdir model && mkdir graph
# Downloading the ai-camp competition dataset
# !wget -N https://ai-camp.s3-us-west-2.amazonaws.com/trainset_11classes_0_00000.zip
# Unzip the data into the folder "data/trainset_11classes_0_00000"
!unzip -qq -n trainset_11classes_0_00000.zip -d data/trainset_11classes_0_00000
# Switch directory to "data/trainset_11classes_0_00000" and show its content
!cd data/trainset_11classes_0_00000 && ls

In [ ]:
import os

base_dir = 'data/trainset_11classes_0_00000'

# Directory to our training data
train_folder = os.path.join(base_dir, 'train')

# Directory to our validation data
val_folder = os.path.join(base_dir, 'val')

# Directory to our model(s)
model_folder = 'model/'

# Directory to our graph(s)
graph_folder = 'graph/'

Now, let's find out the total number of images in each `train` and `val`.

In [ ]:
# List folders and number of files
print("Directory, Number of files")
for root, subdirs, files in os.walk(base_dir):
    print(root, len(files))

We can see that there are 15 categories/folders in each `train` and `val` folder.

Now let's take a look at a few images to get a better sense of what the `KoreanHeart` and `KungfuCrane` categories look like. First, configure the matplotlib parameters:

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

# Index for iterating over images
pic_index = 0

Now, display a batch of 8 KoreanHeart and 8 KungfuCrane poses. You can rerun the cell to see a new batch.

In [ ]:
## Path to KoreanHeart and KungfuCrane
train_koreanheart_dir= "data/trainset_11classes_0_00000/train/KoreanHeart"
train_kungfucrane_dir= "data/trainset_11classes_0_00000/train/KungfuCrane"
train_koreanheart_fnames = os.listdir(train_koreanheart_dir)
train_kungfucrane_fnames = os.listdir(train_kungfucrane_dir)

# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(15, 15)

pic_index += 8
next_koreanheart_pix = [os.path.join(train_koreanheart_dir, fname) 
                for fname in train_koreanheart_fnames[pic_index-8:pic_index]]
next_kungfucrane_pix = [os.path.join(train_kungfucrane_dir, fname) 
                for fname in train_kungfucrane_fnames[pic_index-8:pic_index]]

for i, img_path in enumerate(next_koreanheart_pix+next_kungfucrane_pix):
    # Set up subplot; subplot indices start at 1
    sp = plt.subplot(nrows, ncols, i + 1)
    sp.axis('Off') # Don't show axes (or gridlines)
    
    img = mpimg.imread(img_path)
    plt.imshow(img)

plt.show()

Initialise `train` folder by moving all images from `val` folder to `train` folder.

In [ ]:
import shutil

# Move all images from val folder to train folder
def init_train():
    for root, subdirs, files in os.walk(base_dir):
        if (root.split('/')[-2] == 'val'):
            cls = root.split('/')[-1]
            src_dir = root
            dest_dir = os.path.join(train_folder, cls)
            
            for f in os.listdir(src_dir):
                shutil.move(os.path.join(src_dir, f), dest_dir)
                
init_train()

# Sanity check
# Should see 0 images in all subdirs of data/trainset_11classes_0_00000/val
print("Directory, Number of files")
for root, subdirs, files in os.walk(base_dir):
    print(root, len(files))

Create k groups from images in each class in `train` folder (for k-fold cross validation).

In [ ]:
import random

# l - list, n - size of chunks
def chunks(l, n):
    # For item i in a range that is a length of l,
    for i in range(0, len(l), n):
        # Create an index range for l of n items:
        yield l[i:i+n]

list_dict = []
for x in range(num_classes):
    images = os.listdir(os.path.join(train_folder, classes[x]))
    random.shuffle(images)
    num_val = int(len(images)/k) if k > 1 else int(len(images)*0.2)
    
    i = 0
    dict = {}
    
    for l in list(chunks(images, num_val)):
        for image in l:
            dict[image] = i
        i += 1
        if (i >= k): break

    list_dict.append(dict)
    
# Sanity check
# Should see all images mapped to an index < k for all classes (except when k = 1)
for i, d in enumerate(list_dict):
    print('\033[1m' + str(i+1) + '. ' + classes[i] + '\033[0m')
    print(d)
    print()

Initialise `val` folder by moving designated (index) images from `train` folder to `val` folder.

In [ ]:
# Move designated images from train folder to val folder
def init_val(index):
    init_train()
    
    for x in range(num_classes):
        src_dir = os.path.join(train_folder, classes[x])
        dest_dir = os.path.join(val_folder, classes[x])

        for image, n in list_dict[x].items():
            if (n == index):
                shutil.move(os.path.join(src_dir, image), dest_dir)
        
    for root, subdirs, files in os.walk(base_dir):
        print(root, len(files))

## Data Preprocessing

Let's set up data generators that will read images from our source folders and convert them to float32 tensors. We'll have one generator for each training and validation folders.

### Batch
Our generators will yield batches of `32` images of size `299 x 299` and their labels.

### Feature scaling
Recall that in our MNIST/CIFAR-10 exercises, data that goes into a neural network should be normalised in a way that is easier to be processed by the network. In our case, we will preprocess our images by normalising the pixels values to be in the 0 to 1 range. This happens by dividing each pixel value by 255 and this process is known as data normalisation or rescaling.

### Generator - ImageDataGenerator
To rescale the data, we use `keras.preprocessing.image.ImageDataGenerator` class with the `rescale` parameter. This class will also allow us to instantiate generators of augmented image batches (and their labels) via `.flow_from_directory(directory)`. These generators can then be used with the Keras model methods that accept data generators as inputs such as `fit_generator`, `evaluate_generator` and `predict_generator`. We used data augmentation for the training image generator. To find out more about how to do image augmentation in keras, go [here](https://keras.io/preprocessing/image/).

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import Augmentor

p = None

def augment_data():
    global p
    p = Augmentor.Pipeline(train_folder)
    p.resize(probability=1, width=299, height=299)
    p.random_distortion(probability=0.5, grid_width=4, grid_height=4, magnitude=7)
    p.gaussian_distortion(probability=0.5, grid_width=4, grid_height=4, magnitude=7,corner='bell',method='in')
    p.rotate(probability=0.5, max_left_rotation=15, max_right_rotation=15)
    p.flip_left_right(probability=0.5)
    p.skew(probability=0.5, magnitude=0.25)
    p.shear(probability=0.5, max_shear_left=15, max_shear_right=15)
    p.random_brightness(probability=0.5, min_factor=0.5, max_factor=1.5)
    p.random_contrast(probability=0.5, min_factor=0.75, max_factor=1.25)
    p.status() # sanity check
    
    print("Preparing generator for training dataset")
    train_generator = p.keras_generator(batch_size=bs)
    
    val_datagen = ImageDataGenerator(rescale=1./255)

    # Flow validation images in batches of bs using val_datagen generator
    print("Preparing generator for validation dataset")
    val_generator= val_datagen.flow_from_directory(
        directory= val_folder, 
        target_size=img_size,
        batch_size=bs,
        class_mode='categorical')
    
    return train_generator, val_generator

## Building our CNN Model

The images that will go into our convnet are **299 x 299** color (RGB) images.

Here, we made use of transfer learning. In transfer learning, we take the pre-trained weights of an already trained model (one that has been trained on millions of images belonging to thousands of classes) and use these already learned features to predict new classes in our dataset. We concluded, through multiple experiments, that Xception (by Google) gives us the highest accuracy in classifying our dataset. 

For the classifier block, we went with a [global average pooling layer](https://alexisbcook.github.io/2017/global-average-pooling-layers-for-object-localization/). We also decided not to use a series of dense layers but instead went straight to the final softmax classification layer.

In [ ]:
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.xception import Xception
from keras.applications.densenet import DenseNet201
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.nasnet import NASNetLarge
from keras.models import Model
from keras.layers import Input, Conv2D, BatchNormalization, Activation, GlobalAveragePooling2D, Dense, Dropout, MaxPooling2D, Flatten

def build_model():
    print('Loading model and pre-trained weights...')
    # Specify conv layers
    base_model = Xception(include_top=False, weights='imagenet', pooling=None, input_shape=input_shape)

    for layer in base_model.layers:
        layer.trainable = False
    
    # Blocks 11, 12, 13, 14 to be trained
    for x in range(-36, 0, 1):
        base_model.layers[x].trainable = True
            
    # Specify classifier layers (experiment)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    return Model(inputs=base_model.input, outputs=predictions)

Next, we will configure the specifications for model training.

We train our model with `categorical_crossentropy` loss, because this is a multi-class problem. We will use the `AdaBound` optimizer with default settings. During training, we want to monitor `accuracy` of the classification.

In [ ]:
from keras import optimizers
from adabound import AdaBound

def compile_model(model):
    opt = AdaBound(lr=1e-03,
                   final_lr=0.1,
                   gamma=1e-03,
                   weight_decay=0.,
                   amsbound=False)
    
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])

## Setting Up Checkpoints

Let's setup a [checkpoint](https://keras.io/callbacks/) to help us monitor the validation accuracy as the model trains. This checkpoint will save the model with best validation accuracy seen so far.

## Model Training 

Let's train on all the images in the training set, and validate against all validation images.

Note: This may take a while to run.

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

def train_model(model, train_generator, val_generator, index):
    # Set up checkpoints
    bestValidationCheckpointer = ModelCheckpoint(model_folder + 'train_model_' + str(index) + '.hdf5', 
                                                 monitor='val_acc', 
                                                 save_best_only=True, 
                                                 verbose=1)
    
    earlyStopper = EarlyStopping(monitor='val_acc', 
                                 patience=25,
                                 verbose=1)

    history = model.fit_generator(
          train_generator,
          steps_per_epoch=len(p.augmentor_images) // bs + 1, # train_generator.samples // bs + 1,
          epochs=epochs,
          validation_data=val_generator,
          validation_steps=val_generator.samples // bs + 1,
          callbacks=[bestValidationCheckpointer, earlyStopper])

    return history

## Graphing Accuracy and Loss Functions

Plot accuracy and loss functions.

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, index):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    ep = range(1, len(acc)+1)

    plt.plot(ep, acc, 'bo', label='Training accuracy')
    plt.plot(ep, val_acc, 'b', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.savefig(graph_folder + 'train_acc_' + str(index) + '.png')

    plt.figure()
    plt.plot(ep, loss, 'bo', label='Training loss')
    plt.plot(ep, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.savefig(graph_folder + 'train_loss_' + str(index) + '.png')

    plt.show()

## Evaluating Accuracy and Loss of the Model

With a trained model, we can evaluate the model performance against the truth labels of our validation set. First, we load the best model encountered during training.

Then, we validate accuracy of the loaded model on our good old validation set.

In [ ]:
from keras.models import load_model

def evaluate_model(val_generator, index):
    file = model_folder + 'train_model_' + str(index) + '.hdf5'
    model = load_model(file, custom_objects={'AdaBound': AdaBound})

    val_generator.reset()
    scores = model.evaluate_generator(val_generator, steps=val_generator.samples // val_generator.batch_size + 1, verbose=1)
    os.rename(file, model_folder + 'train_model_' + str(index) + '_' + str(scores[1]) + '.hdf5')
    
    return scores[1]

## Commence Model Training

In [ ]:
avg_acc = 0

for x in range(k):
    init_val(x)
    train_gen, val_gen = augment_data()
    model = build_model()
    model.summary()
    compile_model(model)
    history = train_model(model, train_gen, val_gen, x)
    plot_graphs(history, x)
    avg_acc += evaluate_model(val_gen, x)

avg_acc /= k
print('\033[1m' + 'Overall model\'s accuracy: ' + str(avg_acc) + '\033[0m')